In [1]:
import numpy as np
from redisvl.index import SearchIndex
from redis import Redis
from redisvl.query import VectorQuery, FilterQuery

In [2]:
client = Redis.from_url("redis://localhost:6379")

In [3]:
schema = {
    "index": {
        "name": "user_simple",  # 索引名称
        "prefix": "user_simple_docs",  # 索引键前缀
        "storage_type": "json" # 存储类型(hash或json)
    },
    "fields": [
        {"name": "user", "type": "tag"},  # 参考elasticsearch的keyword类型
        {"name": "job", "type": "text"},  # 参考elasticsearch的text类型
        {"name": "age", "type": "numeric"},  # 参考elasticsearch的number类型
        {
            "name": "user_embedding",
            "type": "vector", 
            "attrs": {
                "dims": 3,  # 向量维度
                "distance_metric": "cosine",  # Metric for measuring query relevance (COSINE, L2, IP).
                "algorithm": "flat",  # Indexing algorithm (flat or hnsw).
                "datatype": "float32"  # Float datatype of the vector (bfloat16, float16, float32, float64).
            }
        }
    ]
}
index = SearchIndex.from_dict(schema, redis_client=client, validate_on_load=True)

In [4]:
index.create(overwrite=True)

In [5]:
data = [
    {
        'user': 'john',
        'age': 1,
        'job': 'engineer',
        'user_embedding': np.array([0.1, 0.1, 0.5], dtype=np.float32)
    },
    {
        'user': 'mary',
        'age': 2,
        'job': 'doctor',
        'user_embedding': np.array([0.2, 0.1, 0.5], dtype=np.float32)
    },
    {
        'user': 'joe',
        'age': 3,
        'job': 'dentist',
        'user_embedding': np.array([0.9, 0.9, 0.1], dtype=np.float32)
    }
]

keys = index.load(data)

# By default, load will create a unique Redis key as a combination of the index key prefix and a random ULID. 
print(keys)

['user_simple_docs:01K38EMZQS24T4N2ZMTEFBFXMB', 'user_simple_docs:01K38EMZS5GRC86ERMD0J51PWS', 'user_simple_docs:01K38EMZTD13C74H1Z62J5BWX4']


In [6]:
query_vector = VectorQuery(
    vector=[0.1, 0.1, 0.5],
    vector_field_name="user_embedding",
    return_fields=["user", "age", "job", "vector_distance", 'user_embedding'],
    num_results=3
)
results_vector = index.query(query_vector)
results_vector

[{'id': 'user_simple_docs:01K38EMZQS24T4N2ZMTEFBFXMB',
  'vector_distance': '0',
  'user': 'john',
  'age': '1',
  'job': 'engineer',
  'user_embedding': '[0.10000000149011612,0.10000000149011612,0.5]'},
 {'id': 'user_simple_docs:01K38EMZS5GRC86ERMD0J51PWS',
  'vector_distance': '0.0161802768707',
  'user': 'mary',
  'age': '2',
  'job': 'doctor',
  'user_embedding': '[0.20000000298023224,0.10000000149011612,0.5]'},
 {'id': 'user_simple_docs:01K38EMZTD13C74H1Z62J5BWX4',
  'vector_distance': '0.653301358223',
  'user': 'joe',
  'age': '3',
  'job': 'dentist',
  'user_embedding': '[0.8999999761581421,0.8999999761581421,0.10000000149011612]'}]

In [7]:
query_filter = FilterQuery(
    filter_expression="*", 
    return_fields=["user", "age", "job", 'user_embedding']
)

results_filter = index.query(query_filter)
results_filter

[{'id': 'user_simple_docs:01K38EMZQS24T4N2ZMTEFBFXMB',
  'user': 'john',
  'age': '1',
  'job': 'engineer',
  'user_embedding': '[0.10000000149011612,0.10000000149011612,0.5]'},
 {'id': 'user_simple_docs:01K38EMZS5GRC86ERMD0J51PWS',
  'user': 'mary',
  'age': '2',
  'job': 'doctor',
  'user_embedding': '[0.20000000298023224,0.10000000149011612,0.5]'},
 {'id': 'user_simple_docs:01K38EMZTD13C74H1Z62J5BWX4',
  'user': 'joe',
  'age': '3',
  'job': 'dentist',
  'user_embedding': '[0.8999999761581421,0.8999999761581421,0.10000000149011612]'}]

In [8]:
# Clear all data from Redis associated with the index
index.clear()

3

In [9]:
# Check if the index exists in Redis.
index.exists()

True

In [10]:
# Remove / delete the index in its entirety
index.delete()

In [11]:
index.exists()

False